In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing as pp
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV
import pickle
import sqlite3

In [ ]:
df = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/users_design_matrix_cont.csv')

In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
# Sim scores
train_set2 = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train_sim.csv')
test_set2 = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test_sim.csv')

y_train_set = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train_sim.csv')
y_test_set = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test_sim.csv')


train_set2


In [ ]:
all_data_raw = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_raw.csv')
all_data_sim = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_sim.csv')

In [ ]:
x_trainfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train_full.csv')
x_testfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test_full.csv')

y_trainfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train_full.csv')
y_testfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test_full.csv')

In [ ]:
x_trainfull.set_index(['user', 'label_id'], inplace=True)
y_trainfull.set_index(['user', 'label_id'], inplace=True)
y_testfull.set_index(['user', 'label_id'], inplace=True)
x_testfull.set_index(['user', 'label_id'], inplace=True)

In [ ]:
x_trainfull

In [ ]:
all_data_sim

In [ ]:
y_trainfull = pd.Series(y_trainfull['rank_y'].values, index=y_trainfull.index)


In [ ]:
all_data

In [ ]:
y_train_set.set_index(['user', 'city_id'], inplace=True)
train_set2.set_index(['user', 'city_id'], inplace=True)
y_test_set.set_index(['user', 'city_id'], inplace=True)
test_set2.set_index(['user', 'city_id'], inplace=True)

In [ ]:
train_set2.drop(columns=['top'], inplace=True)
test_set2.drop(columns=['top'], inplace=True)

In [ ]:
# Raw attraction and food scores
x_test = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test.csv')
y_test = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test.csv')

x_train = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train.csv')
y_train = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train.csv')

x_train

In [ ]:
y_train.set_index(['user', 'label_id'], inplace=True)
x_train.set_index(['user', 'label_id'], inplace=True)
y_test.set_index(['user', 'label_id'], inplace=True)
x_test.set_index(['user', 'label_id'], inplace=True)

## Fake user data

In [ ]:
x_test_fake = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/x_testfake.csv")
y_test_fake = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/y_testfake.csv")

x_train_fake = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/x_trainfake.csv")
y_train_fake = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/y_trainfake.csv")

In [ ]:
x_test_fake.set_index(['user', 'city_id'], inplace=True)
y_test_fake.set_index(['user', 'city_id'], inplace=True)
x_train_fake.set_index(['user', 'city_id'], inplace=True)
y_train_fake.set_index(['user', 'city_id'], inplace=True)

In [ ]:
y_test_fake[y_test_fake['rank'] == 3]

In [ ]:
def rank_from_col(x):
    if x['rank'] == 5:
        return 1
    elif x['rank'] == 4:
        return 1
    elif x['rank'] == 3:
        return 1
    elif x['rank'] == 2:
        return 1
    elif x['rank'] == 1:
        return 1
    else:
        return 0 

In [ ]:
#y_test_fake['top'] = y_test_fake.apply(rank_from_col, axis=1)

y_train_fake['top'] = y_train_fake.apply(rank_from_col, axis=1)


In [ ]:
y_train_fake[y_train_fake['rank'] == 5]

In [ ]:
y_train_fake.drop(columns=['rank'], inplace=True)
y_test_fake.drop(columns=['rank'], inplace=True)

### Model Building

In [ ]:

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=12, max_depth=25)
clf.fit(x_trainfull, y_trainfull)
predictions_rf = clf.predict(x_testfull)
probs_rf = clf.predict_proba(x_testfull)

In [ ]:
pickle.dump(clf, open('model_test.pkl','wb'))

model = pickle.load(open('model_test.pkl','rb'))
predicts = model.predict(x_user)
probs = model.predict_proba(x_user)


In [ ]:
probs

In [ ]:
x_user

In [ ]:
y_trainfull = pd.Series(y_trainfull['rank_y'].values, index=y_trainfull.index)


In [ ]:
print(classification_report(y_testfull, predictions_rf))
print(confusion_matrix(y_testfull,predictions_rf))

In [ ]:
from sklearn.metrics import mean_squared_error 
#print('RMSE on train data: ', mean_squared_error(y_trainfull, )**(0.5))
print('RMSE on test data: ',  mean_squared_error(y_testfull, predictions_rf)**(0.5))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,7))
feat_importances = pd.Series(clf.feature_importances_, index = x_trainfull.columns)
feat_importances.nlargest(7).plot(kind='barh');

In [ ]:
x_testfull.index.get_level_values('user').unique()

In [ ]:
x_user = x_testfull.xs(55, level='user', drop_level=False).copy()
y_user = y_testfull.xs(55, level='user', drop_level=False).copy()

### Accuracy Score Metric

In [ ]:
results5 = x_testfull.copy()

In [ ]:
results = x_user.copy()

In [ ]:
# This will add the probability and the predicted values from the model 
def split_array(prob, x_df, predict):
    df = x_df.copy()
    lst1 = list(prob[:, 0])
    lst2 = list(prob[:, 1])
    predict_lst = list(predict)
    df['prob_0'] = pd.Series(lst1, index=df.index)
    df['prob_1'] = pd.Series(lst2, index=df.index)
    df['predicted'] = pd.Series(predict_lst, index=df.index)
    
    
    
    return df

In [ ]:
results

In [ ]:
okk = split_array(probs, results, predicts)
okk

In [ ]:
ok.sort_values('prob_1', ascending=False)

In [ ]:
cities = get_df('cities')
cities

In [ ]:
def add_top_city(results_df, y_test_set):
    
    # adds actual top ranked cities
    results_df_top = pd.merge(right=y_test_set, left=results_df, right_index=True, left_index=True)
    
    return results_df_top

def add_city_name(results_df, city_df):
    results_df.reset_index(level=1, inplace=True)
    
    city_dict = dict(zip(city_df['city_id'], city_df['city']))
    results_df['city'] = results_df['label_id'].map(city_dict)
    results_df.set_index('label_id', append=True, inplace=True)
    
    return results_df

In [ ]:
df.columns

In [ ]:
ok2 = add_city_name(okk, df)
ok2

In [ ]:
ok3 = add_top_city(ok2, y_user)

In [ ]:
ok3

In [ ]:
ok3.sort_values('prob_1', ascending=False)

In [ ]:
# filter only continent = 4 and rank_y != 0 
# Choose and print the top 3 cities

In [ ]:
y_user